# Exploracion de datos sobre cobertura movil en Colombia

In [130]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import math

In [131]:
df_cobertura_movil = pd.read_csv('data/Cobertura_móvil_por_tecnología,_departamento_y_municipio_por_proveedor_20251031.csv')
df_cobertura_movil.sample(5)


,AÑO,TRIMESTRE,PROVEEDOR,COD DEPARTAMENTO,DEPARTAMENTO,COD MUNICIPIO,MUNICIPIO,CABECERA MUNICIPAL,COD CENTRO POBLADO,CENTRO POBLADO,COBERTURA 2G,COBERTURA 3G,"COBERTURA HSPA+, HSPA+DC",COBERTUTA 4G,COBERTURA LTE,COBERTURA 5G
269402,2017,2,AVANTEL S.A.S,86,PUTUMAYO,86571.0,PUERTO GUZMÁN,S,86571001.0,PUERTO GUZMÁN,N,N,N,N,N,N
82346,2022,1,COLOMBIA TELECOMUNICACIONES S.A. E.S.P.,25,CUNDINAMARCA,25299.0,GAMA,S,25299000.0,GAMA,S,S,S,N,N,N
200823,2018,3,COMUNICACION CELULAR S A COMCEL S A,68,SANTANDER,68160.0,CEPITÁ,S,68160001.0,CEPITÁ,S,S,S,N,N,N
297755,2015,4,COMUNICACION CELULAR S A COMCEL S A,68,SANTANDER,68190.0,CIMITARRA,N,68190021.0,PALMAS DEL GUAYABITO,N,N,N,N,N,N
226220,2017,1,COLOMBIA MOVIL S.A ESP,76,VALLE DEL CAUCA,76622.0,ROLDANILLO,S,76622001.0,ROLDANILLO,S,S,S,N,N,N


In [132]:
df_datos_geograficos = pd.read_csv(
    'data/DIVIPOLA_CentrosPoblados.csv', 
    encoding='latin-1', 
    sep=';'
)
df_datos_geograficos.sample(5)


,Código_Departamento,Nombre_Departamento,Código_Municipio,Nombre_Municipio,Código_Entidad,Nombre_Entidad,Tipo,Longitud,Latitud
4068,27,CHOCÓ,27150,CARMEN DEL DARIÉN,27150001,BRISAS,CP,"-76,766464","7,315101"
4034,27,CHOCÓ,27077,BAJO BAUDÓ,27077046,EL LLANO,CP,"-77,1076286","5,28454907"
5326,52,NARIÑO,52110,BUESACO,52110007,VILLAMORENO,CP,"-77,199005","1,323575"
7235,73,TOLIMA,73622,RONCESVALLES,73622002,EL CEDRO,CP,"-75,458761","4,068463"
4370,41,HUILA,41244,ELÍAS,41244002,ORITOGUAZ,CP,"-76,012499","1,986866"


In [133]:
cobertura_clean = df_cobertura_movil.drop_duplicates().copy()

cobertura_clean['COD CENTRO POBLADO'] = cobertura_clean['COD CENTRO POBLADO'].fillna(0).astype(int)
cobertura_clean['COD MUNICIPIO'] = cobertura_clean['COD MUNICIPIO'].fillna(0).astype(int)

divipola_clean = df_datos_geograficos.copy()
divipola_clean['Longitud'] = divipola_clean['Longitud'].str.replace(',', '.').astype(float)
divipola_clean['Latitud'] = divipola_clean['Latitud'].str.replace(',', '.').astype(float)

df = cobertura_clean.merge(
    divipola_clean, 
    left_on='COD CENTRO POBLADO', 
    right_on='Código_Entidad', 
    how='left'
)
df.to_csv('data/cobertura_movil_con_datos_geograficos.csv', index=False)
df.head()

,AÑO,TRIMESTRE,PROVEEDOR,COD DEPARTAMENTO,DEPARTAMENTO,COD MUNICIPIO,MUNICIPIO,CABECERA MUNICIPAL,COD CENTRO POBLADO,CENTRO POBLADO,...,COBERTURA 5G,Código_Departamento,Nombre_Departamento,Código_Municipio,Nombre_Municipio,Código_Entidad,Nombre_Entidad,Tipo,Longitud,Latitud
0,2023,3,COLOMBIA MOVIL S.A ESP,27,CHOCÓ,27250,EL LITORAL DEL SAN JUAN,N,27250034,TORDÓ,...,N,27.0,CHOCÓ,27250.0,EL LITORAL DEL SAN JUAN,27250034.0,TORDÓ,CP,-77.016686,4.337732
1,2023,3,COLOMBIA TELECOMUNICACIONES S.A. E.S.P.,5,ANTIOQUIA,5495,NECHÍ,N,5495003,LA CONCHA,...,N,5.0,ANTIOQUIA,5495.0,NECHÍ,5495003.0,LA CONCHA,CP,-74.868204,7.948383
2,2022,3,COLOMBIA MOVIL S.A ESP,70,SUCRE,70508,OVEJAS,N,70508006,DON GABRIEL,...,N,70.0,SUCRE,70508.0,OVEJAS,70508006.0,DON GABRIEL,CP,-75.293051,9.623491
3,2021,4,AVANTEL S.A.S,73,TOLIMA,73043,ANZOÁTEGUI,S,73043000,ANZOÁTEGUI,...,N,73.0,TOLIMA,73043.0,ANZOÁTEGUI,73043000.0,ANZOÁTEGUI,CM,-75.093772,4.631756
4,2021,2,COMUNICACION CELULAR S A COMCEL S A,50,META,50150,CASTILLA LA NUEVA,N,50150001,SAN LORENZO,...,N,50.0,META,50150.0,CASTILLA LA NUEVA,50150001.0,SAN LORENZO,CP,-73.548038,3.811495


In [134]:
def obtener_crecimiento_de_cobertura_por_ano(df, tecnologia):
	df_tecnologia = df[df[tecnologia] == 'S']
	df_tecnologia_por_ano = df_tecnologia.groupby('AÑO').size().reset_index(name='Cantidad de centros poblados con cobertura')
	return df_tecnologia_por_ano
	
tipos_de_tecnologia = [
    'COBERTURA 2G',
    'COBERTURA 3G',
    'COBERTURA HSPA+, HSPA+DC',
    'COBERTUTA 4G',
    'COBERTURA LTE',
    'COBERTURA 5G'
]
df_crecimiento_cobertura = {}

for tecnologia in tipos_de_tecnologia:
	df_crecimiento_cobertura[tecnologia] = obtener_crecimiento_de_cobertura_por_ano(df, tecnologia)
	
fig = go.Figure()

for tecnologia, df_tecnologia in df_crecimiento_cobertura.items():
	fig.add_trace(
		go.Scatter(
			x=df_tecnologia['AÑO'], 
			y=df_tecnologia['Cantidad de centros poblados con cobertura'], 
			mode='lines+markers', 
			name=tecnologia
		)
	)
fig.update_layout(
	title='Crecimiento de cobertura móvil por tecnología en Colombia',
	xaxis_title='Año',
	yaxis_title='Cantidad de centros poblados con cobertura',
	legend_title='Tecnología'
)
fig.write_html('graficos/crecimiento_cobertura_movil_por_tecnologia.html')
fig.show()



In [135]:
def obtener_velocidad_de_adopcion_de_tecnologia_por_proveedor(df, tecnologia, proveedor):
	df_tecnologia_proveedor = df[(df[tecnologia] == 'S') & (df['PROVEEDOR'] == proveedor)]
	df_tecnologia_proveedor_por_ano = df_tecnologia_proveedor.groupby('AÑO').size().reset_index(name='Cantidad de centros poblados con cobertura')
	return df_tecnologia_proveedor_por_ano


def mostrar_velocidad_adopcion_tecnologia_por_proveedor(df, tecnologia, proveedor):
	df_adopcion_tecnologia_2g = {}
	fig = go.Figure()
	for proveedor in proveedores:
		df_adopcion_tecnologia_2g[proveedor] = obtener_velocidad_de_adopcion_de_tecnologia_por_proveedor(df, 'COBERTURA 2G', proveedor)
	fig = go.Figure()
	for proveedor, df_proveedor in df_adopcion_tecnologia_2g.items():
		fig.add_trace(
			go.Scatter(
				x=df_proveedor['AÑO'], 
				y=df_proveedor['Cantidad de centros poblados con cobertura'], 
				mode='lines+markers', 
				name=proveedor
			)
		)
	fig.update_layout(
		title=f'Velocidad de adopción de {tecnologia} por {proveedor} en Colombia',
		xaxis_title='Año',
		yaxis_title='Cantidad de centros poblados con cobertura',
		legend_title='Proveedor'
	)
	tecnologia_clean = tecnologia.replace(' ', '_').lower()
	proveedor_clean = proveedor.replace(' ', '_').lower()
	fig.write_html(f'graficos/velocidad_adopcion_tecnologia_{tecnologia_clean}_por_{proveedor_clean}.html')
	fig.show()	


In [136]:
def mostrar_velocidad_adopcion_con_dropdown(df, tecnologias, proveedores):
    fig = go.Figure()
    
    for tecnologia in tecnologias:
        for proveedor in proveedores:
            df_data = obtener_velocidad_de_adopcion_de_tecnologia_por_proveedor(
                df, tecnologia, proveedor
            )
            fig.add_trace(
                go.Scatter(
                    x=df_data['AÑO'],
                    y=df_data['Cantidad de centros poblados con cobertura'],
                    mode='lines+markers',
                    name=proveedor,
                    visible=(tecnologia == tecnologias[0])  # Solo la primera visible
                )
            )
    
    buttons = []
    traces_per_tech = len(proveedores)
    
    for i, tecnologia in enumerate(tecnologias):
        visibility = [False] * len(fig.data)
        for j in range(i * traces_per_tech, (i + 1) * traces_per_tech):
            visibility[j] = True
        
        buttons.append(
            dict(
                label=tecnologia,
                method='update',
                args=[{'visible': visibility},
                      {'title': f'Adopción de {tecnologia} por proveedor'}]
            )
        )
    
    fig.update_layout(
        updatemenus=[
            dict(
                buttons=buttons,
                direction='down',
                showactive=True,
                x=0.17,
                y=1.15
            )
        ],
        title=f'Adopción de {tecnologias[0]} por proveedor',
        xaxis_title='Año',
        yaxis_title='Centros poblados con cobertura'
    )
    
    fig.write_html('graficos/velocidad_adopcion_interactiva.html')
    fig.show()

proveedores = df['PROVEEDOR'].unique()
mostrar_velocidad_adopcion_con_dropdown(df, tipos_de_tecnologia, proveedores)
 

## Análisis de competencia entre proveedores

Cuota de mercado de cada proveedor por tecnología a lo largo del tiempo.

In [137]:
año_reciente = df['AÑO'].max()

todas_tecnologias = [col for col in df.columns if 'COBERTURA' in col or 'COBERTUTA' in col]


n_tecnologias = len(todas_tecnologias)
n_cols = 3 
n_rows = math.ceil(n_tecnologias / n_cols)

nombres_tecnologias = []
for tech in todas_tecnologias:
    nombre = tech.replace('COBERTURA ', '').replace('COBERTUTA ', '')
    nombres_tecnologias.append(nombre)

specs = [[{'type':'domain'} for _ in range(n_cols)] for _ in range(n_rows)]

fig = make_subplots(
    rows=n_rows, cols=n_cols,
    specs=specs,
    subplot_titles=nombres_tecnologias
)

for idx, tecnologia in enumerate(todas_tecnologias):
    row = (idx // n_cols) + 1
    col = (idx % n_cols) + 1
    
    cuota = analizar_cuota_mercado_por_tecnologia(df, tecnologia, año_reciente)
    
    fig.add_trace(
        go.Pie(
            labels=cuota['PROVEEDOR'],
            values=cuota['Centros con cobertura'],
            name=nombres_tecnologias[idx],
            hovertemplate='%{label}<br>%{value} centros<br>%{percent}<extra></extra>'
        ),
        row=row, col=col
    )

fig.update_layout(
    title_text=f'Cuota de Mercado por Proveedor - Todas las Tecnologías ({año_reciente})',
    height=400 * n_rows,
    showlegend=True
)

fig.write_html('graficos/cuota_mercado_todas_tecnologias.html')
fig.show()

In [138]:
todas_tecnologias = [col for col in df.columns if 'COBERTURA' in col or 'COBERTUTA' in col]

fig = go.Figure()

todos_proveedores = df['PROVEEDOR'].unique()

for idx, tecnologia in enumerate(todas_tecnologias):
    df_evolucion = analizar_evolucion_cuota_mercado(df, tecnologia)
    
    if len(df_evolucion) == 0:
        continue
    
    for proveedor in todos_proveedores:
        df_proveedor = df_evolucion[df_evolucion['PROVEEDOR'] == proveedor]
        
        if len(df_proveedor) > 0:
            fig.add_trace(
                go.Scatter(
                    x=df_proveedor['AÑO'],
                    y=df_proveedor['Porcentaje'],
                    name=proveedor,
                    mode='lines',
                    stackgroup='one', 
                    visible=(idx == 0),
                    hovertemplate='%{y:.1f}%<br>%{customdata} centros<extra></extra>',
                    customdata=df_proveedor['Centros']
                )
            )

buttons = []
traces_por_tecnologia = []

idx_actual = 0
for tecnologia in todas_tecnologias:
    df_evolucion = analizar_evolucion_cuota_mercado(df, tecnologia)
    if len(df_evolucion) == 0:
        traces_por_tecnologia.append((idx_actual, idx_actual))
        continue
    
    n_proveedores_con_datos = 0
    for proveedor in todos_proveedores:
        if len(df_evolucion[df_evolucion['PROVEEDOR'] == proveedor]) > 0:
            n_proveedores_con_datos += 1
    
    traces_por_tecnologia.append((idx_actual, idx_actual + n_proveedores_con_datos))
    idx_actual += n_proveedores_con_datos

for idx, tecnologia in enumerate(todas_tecnologias):
    nombre_limpio = tecnologia.replace('COBERTURA ', '').replace('COBERTUTA ', '')
    
    visibility = [False] * len(fig.data)
    inicio, fin = traces_por_tecnologia[idx]
    for i in range(inicio, fin):
        visibility[i] = True
    
    buttons.append(
        dict(
            label=nombre_limpio,
            method='update',
            args=[
                {'visible': visibility},
                {'title': f'Evolución de Cuota de Mercado en {nombre_limpio} por Proveedor'}
            ]
        )
    )

primera_tech = todas_tecnologias[0].replace('COBERTURA ', '').replace('COBERTUTA ', '')

fig.update_layout(
    updatemenus=[
        dict(
            buttons=buttons,
            direction='down',
            showactive=True,
            x=0.17,
            y=1.15,
            xanchor='left',
            yanchor='top'
        )
    ],
    title=f'Evolución de Cuota de Mercado en {primera_tech} por Proveedor',
    xaxis_title='Año',
    yaxis_title='Cuota de Mercado (%)',
    hovermode='x unified',
    height=600
)

fig.write_html('graficos/evolucion_cuota_mercado_interactiva.html')
fig.show()

In [139]:
def identificar_lideres_mercado(df):
    años = sorted(df['AÑO'].unique())
    lideres = []
    
    for tecnologia in tecnologias_para_analizar:
        for año in años:
            try:
                cuota = analizar_cuota_mercado_por_tecnologia(df, tecnologia, año)
                if len(cuota) > 0:
                    lider = cuota.iloc[0]
                    lideres.append({
                        'AÑO': año,
                        'Tecnología': tecnologia.replace('COBERTURA ', '').replace('COBERTUTA ', ''),
                        'Líder': lider['PROVEEDOR'],
                        'Cuota (%)': lider['Porcentaje']
                    })
            except:
                pass
    
    return pd.DataFrame(lideres)

df_lideres = identificar_lideres_mercado(df)

for tecnologia in df_lideres['Tecnología'].unique():
    df_tech = df_lideres[df_lideres['Tecnología'] == tecnologia]

fig = px.bar(
    df_lideres,
    x='AÑO',
    y='Cuota (%)',
    color='Líder',
    facet_col='Tecnología',
    title='Cuota del Líder de Mercado por Tecnología',
    labels={'Cuota (%)': 'Cuota de Mercado del Líder (%)'}
)

fig.update_layout(height=400)
fig.write_html('graficos/lideres_mercado_por_tecnologia.html')
fig.show()